In [11]:
!pwd

/home/yutankim/Desktop/EfficientDet_TensorFlow2


In [1]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing


def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

#모델 구조
model = EfficientDet()
print_model_summary(model)

#weight 불러오기
load_weights_from_epoch = Config.load_weights_from_epoch_quan
model.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))
#model.load_weights(filepath=Config.save_model_dir+"saved_model")

2021-10-25 18:54:11.933829: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-25 18:54:12.485360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:3b:00.0, compute capability: 7.5
2021-10-25 18:54:13.632722: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2021-10-25 18:54:14.288179: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "efficient_det"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficient_net (EfficientNet) multiple                  6771296   
_________________________________________________________________
bi_fpn (BiFPN)               multiple                  129126    
_________________________________________________________________
box_class_predict (BoxClassP multiple                  249069    
Total params: 7,149,491
Trainable params: 7,106,515
Non-trainable params: 42,976
_________________________________________________________________


In [30]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
print(dataset_list)
image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
image = tf.expand_dims(image, 0)
print(image)
# 여기 부분, 기존 코드의 data_preprocesssing 과 동일하게 맞추기
def representative_data_gen():
    dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
    for i in range(100):
        image = next(iter(dataset_list))
        image = tf.io.read_file(image)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (512,512))
        # image = tf.cast(image / 255., tf.float32)
        image = tf.expand_dims(image, 0)
        print(image)
# Model has only one input so each data point has one element
        yield [image]

#     #### 수정좀 해보기
# def representative_data_gen():
#   dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
#   for i in range(100):
#     image = next(iter(dataset_list))
#     image = tf.io.read_file(image)
#     image = tf.io.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (512,512))
#     # image = tf.cast(image / 255., tf.float32)
#     image = tf.expand_dims(image, 0)
# # Model has only one input so each data point has one element
#     yield [image]

<ShuffleDataset shapes: (), types: tf.string>
tf.Tensor(
[[[[ 86.75      87.75      69.75    ]
   [ 98.125     99.5       80.375   ]
   [ 85.        85.75      70.375   ]
   ...
   [ 97.        97.        97.      ]
   [ 96.375     96.375     96.375   ]
   [ 96.        96.        96.      ]]

  [[ 84.9375    85.9375    69.52344 ]
   [ 85.55078   86.92578   66.78125 ]
   [113.77344  115.65625   94.16406 ]
   ...
   [ 98.47266   98.47266   98.47266 ]
   [ 97.28125   97.28125   97.28125 ]
   [ 96.90625   96.90625   96.90625 ]]

  [[ 96.66797   96.92969   83.63281 ]
   [130.55078  132.5586   109.46094 ]
   [135.6289   139.84375  104.85156 ]
   ...
   [ 99.78516   99.78516   99.78516 ]
   [ 99.0625    99.0625    99.0625  ]
   [ 98.6875    98.6875    98.6875  ]]

  ...

  [[ 57.933594  57.933594  57.933594]
   [ 70.77734   70.77734   70.77734 ]
   [ 84.234375  84.234375  84.234375]
   ...
   [108.41406  108.41406  108.41406 ]
   [105.38672  105.38672  105.38672 ]
   [ 99.72266   99.72266   9

In [31]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model_quant = converter.convert()

2021-10-25 19:21:41.428039: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-10-25 19:21:41.428163: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-10-25 19:21:41.430036: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:3b:00.0, compute capability: 7.5
2021-10-25 19:21:41.544566: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.018ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.



tf.Tensor(
[[[[26.875     25.875      4.875    ]
   [50.625     49.625     28.625    ]
   [32.5       31.5       10.5      ]
   ...
   [ 5.         5.         5.       ]
   [ 5.         5.         5.       ]
   [ 4.125      4.125      4.125    ]]

  [[41.48828   40.48828   20.394531 ]
   [39.75      38.75      18.65625  ]
   [44.847656  43.847656  23.753906 ]
   ...
   [ 5.         5.         5.       ]
   [ 5.         5.         5.       ]
   [ 4.125      4.125      4.125    ]]

  [[15.15625   14.15625    3.4375   ]
   [17.742188  16.742188   6.8671875]
   [31.570312  30.570312  12.2578125]
   ...
   [ 5.         5.         5.       ]
   [ 5.         5.         5.       ]
   [ 4.125      4.125      4.125    ]]

  ...

  [[ 5.125      5.125      5.125    ]
   [ 6.         6.         6.       ]
   [ 6.         6.         6.       ]
   ...
   [ 6.         6.         6.       ]
   [ 6.         6.         6.       ]
   [ 6.         6.         6.       ]]

  [[ 5.125      5.125      5.125  

2021-10-25 19:21:44.264158: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-10-25 19:21:44.264200: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-10-25 19:21:44.725178: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1899] Estimated count of arithmetic ops: 235.640 G  ops, equivalently 117.820 G  MACs


RuntimeError: tensorflow/lite/kernels/reshape.cc:69 num_input_elements != num_output_elements (576 != 512)Node number 503 (RESHAPE) failed to prepare.


In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

In [ ]:
with open('./tflite_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [ ]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print(output_details)

In [ ]:
model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    return interpreter

def prediction(model, data):
    interpreter = model
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    input_data = np.array(data, dtype = np.uint8)
    interpreter.set_tensor(input_details['index'], input_data)
    interpreter.invoke()
    output= interpreter.get_tensor(output_details['index'])
    return output

In [ ]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')

image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
# image = tf.cast(image / 255., tf.float32)
image = tf.expand_dims(image, 0)
print(image.shape)

In [ ]:
import numpy as np

model = load_model(model_path)
output = prediction(model, image)

In [ ]:
print(output.shape)
print(output)